In [137]:
import streamlit as st
from datetime import date
import yfinance as yf
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go

In [138]:
START="2020-01-01"
TODAY=date.today().strftime("%Y-%m-%d")

In [139]:
st.title("Stock Prediction App")

2024-12-17 13:33:20.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [140]:
stocks=("AAPL","GOOG","MSFT")
selected_stock=st.selectbox("Select dataset for prediction",stocks)

2024-12-17 13:33:20.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.235 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.237 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [141]:
n_years=st.slider("years of prediction:",1,4)

2024-12-17 13:33:20.252 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.261 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.265 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [142]:
period=n_years*365

In [143]:
def load_data(ticker):
    data=yf.download(ticker,START,TODAY)
    data.reset_index(inplace=True)
    return data

In [144]:
# @title
data_load_state=st.text("Load data...")
data=load_data(selected_stock)
data_load_state.text("Loading data...done!")

2024-12-17 13:33:20.314 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
[*********************100%***********************]  1 of 1 completed
2024-12-17 13:33:20.444 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [145]:
data.head(3)

Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
0,2020-01-02,72.796021,75.087502,75.150002,73.797501,74.059998,135480400
1,2020-01-03,72.088295,74.357498,75.144997,74.125000,74.287498,146322800
2,2020-01-06,72.662712,74.949997,74.989998,73.187500,73.447502,118387200


In [146]:
st.subheader('Raw Data')
st.write(data.tail(5))

2024-12-17 13:33:20.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.545 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.548 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [147]:
def plot_raw_data():
  fig=go.Figure()
  fig.add_trace(go.Scatter(x=data['Date'],y=data['Open'],name="stock_open"))
  fig.add_trace(go.Scatter(x=data['Date'],y=data['Close'],name="stock_close"))
  fig.layout.update(title_text="Time Series Data with slider",xaxis_rangeslider_visible=True)
  st.plotly_chart(fig)


In [148]:
plot_raw_data()

2024-12-17 13:33:20.666 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:20.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [149]:
import pandas as pd

In [150]:
df_train=data[['Date','Close']]
df_train = data[['Date', 'Close']].rename(columns={"Date": "ds", "Close": "y"})


In [151]:
# Flatten multi-level column names
data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]

# Check the updated columns
print("Flattened Columns:", data.columns)


Flattened Columns: Index(['Date_', 'Adj Close_AAPL', 'Close_AAPL', 'High_AAPL', 'Low_AAPL',
       'Open_AAPL', 'Volume_AAPL'],
      dtype='object')


In [152]:
# Create df_train with correct flattened column names
df_train = data[['Date_', 'Close_AAPL']].rename(columns={"Date_": "ds", "Close_AAPL": "y"})
print(df_train.head())


          ds          y
0 2020-01-02  75.087502
1 2020-01-03  74.357498
2 2020-01-06  74.949997
3 2020-01-07  74.597504
4 2020-01-08  75.797501


In [153]:
m=Prophet()
m.fit(df_train)
future=m.make_future_dataframe(periods=period)
forecast=m.predict(future)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdb7uvego/ddi6anty.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdb7uvego/aonurqik.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28681', 'data', 'file=/tmp/tmpdb7uvego/ddi6anty.json', 'init=/tmp/tmpdb7uvego/aonurqik.json', 'output', 'file=/tmp/tmpdb7uvego/prophet_model2hx4yi_g/prophet_model-20241217133321.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:33:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:33:23 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [154]:
st.subheader('Forecast Data')
st.write(forecast.tail())

2024-12-17 13:33:24.149 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.169 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [155]:
st.write(f'Forecast plot for {n_years} years')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

2024-12-17 13:33:24.190 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.345 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [156]:
st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)

2024-12-17 13:33:24.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:24.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:25.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:27.601 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 13:33:27.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
